In [1]:
import os, sys
import paddle
sys.path.append('/workspace/fnet_paddle/PaddleNLP')
from paddlenlp.datasets import load_dataset

In [2]:
train_ds, dev_ds = load_dataset("glue", name="cola", splits=("train", "dev"))

INFO:paddle.utils.download:unique_endpoints {''}


In [3]:
train_ds[:3], dev_ds[:3]

([{'sentence': "Our friends won't buy this analysis, let alone the next one we propose.",
   'labels': 1},
  {'sentence': "One more pseudo generalization and I'm giving up.",
   'labels': 1},
  {'sentence': "One more pseudo generalization or I'm giving up.",
   'labels': 1}],
 [{'sentence': 'The sailors rode the breeze clear of the rocks.', 'labels': 1},
  {'sentence': 'The weights made the rope stretch over the pulley.',
   'labels': 1},
  {'sentence': 'The mechanical doll wriggled itself loose.', 'labels': 1}])

In [4]:
def convert_example(example,
                    tokenizer,
                    max_seq_length=512,
                    is_test=False):
    text = example["sentence"]
    text_pair = None
    encoded_inputs = tokenizer(
        text=text, text_pair=text_pair, max_seq_len=max_seq_length)
    input_ids = encoded_inputs["input_ids"]
    token_type_ids = encoded_inputs["token_type_ids"]

    if is_test:
        return input_ids, token_type_ids
    label = np.array([example["labels"]], dtype="int64")
    return input_ids, token_type_ids, label

In [5]:
from functools import partial
import argparse
import os
import random
import time
import distutils.util
import numpy as np
import paddle
import paddle.nn.functional as F
import paddlenlp as ppnlp
from paddlenlp.data import Stack, Tuple, Pad
from paddlenlp.datasets import load_dataset
from paddlenlp.transformers import LinearDecayWithWarmup

In [6]:
parser = argparse.ArgumentParser()
parser.add_argument("--save_dir", default='./checkpoint', type=str, help="The output directory where the model checkpoints will be written.")
parser.add_argument("--max_seq_length", default=128, type=int, help="The maximum total input sequence length after tokenization. "
    "Sequences longer than this will be truncated, sequences shorter will be padded.")
parser.add_argument("--batch_size", default=32, type=int, help="Batch size per GPU/CPU for training.")
parser.add_argument("--learning_rate", default=5e-5, type=float, help="The initial learning rate for Adam.")
parser.add_argument("--weight_decay", default=0.0, type=float, help="Weight decay if we apply some.")
parser.add_argument("--epochs", default=3, type=int, help="Total number of training epochs to perform.")
parser.add_argument("--warmup_proportion", default=0.0, type=float, help="Linear warmup proption over the training process.")
parser.add_argument("--valid_steps", default=100, type=int, help="The interval steps to evaluate model performance.")
parser.add_argument("--save_steps", default=100, type=int, help="The interval steps to save checkppoints.")
parser.add_argument("--logging_steps", default=10, type=int, help="The interval steps to logging.")
parser.add_argument("--init_from_ckpt", type=str, default=None, help="The path of checkpoint to be loaded.")
parser.add_argument("--seed", type=int, default=1000, help="random seed for initialization")
parser.add_argument('--device', choices=['cpu', 'gpu', 'xpu', 'npu'], default="gpu", help="Select which device to train model, defaults to gpu.")
parser.add_argument("--use_amp", type=distutils.util.strtobool, default=False, help="Enable mixed precision training.")
parser.add_argument("--scale_loss", type=float, default=2**15, help="The value of scale_loss for fp16.")
args = parser.parse_args([])

In [7]:
args.seed = 999
args.batch_size = 64
args.epochs = 3
args.learning_rate=2.5e-03

In [12]:
def set_seed(seed):
    """sets random seed"""
    random.seed(seed)
    np.random.seed(seed)
    paddle.seed(seed)


@paddle.no_grad()
def evaluate(model, criterion, metric, data_loader):
    """
    Given a dataset, it evals model and computes the metric.
    Args:
        model(obj:`paddle.nn.Layer`): A model to classify texts.
        data_loader(obj:`paddle.io.DataLoader`): The dataset loader which generates batches.
        criterion(obj:`paddle.nn.Layer`): It can compute the loss.
        metric(obj:`paddle.metric.Metric`): The evaluation metric.
    """
    model.eval()
    metric.reset()
    losses = []
    for batch in data_loader:
        input_ids, token_type_ids, labels = batch
        logits = model(input_ids, token_type_ids)
        loss = criterion(logits, labels)
        losses.append(loss.numpy())
        correct = metric.compute(logits, labels)
        metric.update(correct)
    accu = metric.accumulate()
    print("eval loss: %.5f, accu: %.5f" % (np.mean(losses), accu))
    model.train()
    metric.reset()


def create_dataloader(dataset,
                      mode='train',
                      batch_size=1,
                      batchify_fn=None,
                      trans_fn=None):
    if trans_fn:
        dataset = dataset.map(trans_fn)

    shuffle = True if mode == 'train' else False
    if mode == 'train':
        batch_sampler = paddle.io.DistributedBatchSampler(
            dataset, batch_size=batch_size, shuffle=shuffle)
    else:
        batch_sampler = paddle.io.BatchSampler(
            dataset, batch_size=batch_size, shuffle=shuffle)

    return paddle.io.DataLoader(
        dataset=dataset,
        batch_sampler=batch_sampler,
        collate_fn=batchify_fn,
        return_list=True)


def do_train():
    paddle.set_device(args.device)
    rank = paddle.distributed.get_rank()
    if paddle.distributed.get_world_size() > 1:
        paddle.distributed.init_parallel_env()

    set_seed(args.seed)

    train_ds, dev_ds = load_dataset("glue", name="cola", splits=("train", "dev"))

    fnet = ppnlp.transformers.FNetModel.from_pretrained('pretrained_model/paddle/large')
    model = ppnlp.transformers.FNetForSequenceClassification(fnet, num_classes=len(train_ds.label_list))
    tokenizer = ppnlp.transformers.FNetTokenizer.from_pretrained('fnet-large')

    trans_func = partial(
        convert_example,
        tokenizer=tokenizer,
        max_seq_length=args.max_seq_length)
    batchify_fn = lambda samples, fn=Tuple(
        Pad(axis=0, pad_val=tokenizer.pad_token_id),  # input
        Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # segment
        Stack(dtype="int64")  # label
    ): [data for data in fn(samples)]
    train_data_loader = create_dataloader(
        train_ds,
        mode='train',
        batch_size=args.batch_size,
        batchify_fn=batchify_fn,
        trans_fn=trans_func)
    dev_data_loader = create_dataloader(
        dev_ds,
        mode='dev',
        batch_size=args.batch_size,
        batchify_fn=batchify_fn,
        trans_fn=trans_func)

    if args.init_from_ckpt and os.path.isfile(args.init_from_ckpt):
        state_dict = paddle.load(args.init_from_ckpt)
        model.set_dict(state_dict)
    model = paddle.DataParallel(model)

    num_training_steps = len(train_data_loader) * args.epochs

    lr_scheduler = LinearDecayWithWarmup(args.learning_rate, num_training_steps,
                                         args.warmup_proportion)

    # Generate parameter names needed to perform weight decay.
    # All bias and LayerNorm parameters are excluded.
    decay_params = [
        p.name for n, p in model.named_parameters()
        if not any(nd in n for nd in ["bias", "norm"])
    ]
    optimizer = paddle.optimizer.AdamW(
        learning_rate=lr_scheduler,
        parameters=model.parameters(),
        weight_decay=args.weight_decay,
        apply_decay_param_fun=lambda x: x in decay_params)

    criterion = paddle.nn.loss.CrossEntropyLoss()
    metric = paddle.metric.Accuracy()
    if args.use_amp:
        scaler = paddle.amp.GradScaler(init_loss_scaling=args.scale_loss)
    global_step = 0
    tic_train = time.time()
    for epoch in range(1, args.epochs + 1):
        for step, batch in enumerate(train_data_loader, start=1):
            input_ids, token_type_ids, labels = batch
            with paddle.amp.auto_cast(
                    args.use_amp,
                    custom_white_list=["layer_norm", "softmax", "gelu"], ):
                logits = model(input_ids, token_type_ids)
                loss = criterion(logits, labels)
            probs = F.softmax(logits, axis=1)
            correct = metric.compute(probs, labels)
            metric.update(correct)
            acc = metric.accumulate()

            global_step += 1
            if global_step % args.logging_steps == 0 and rank == 0:
                print(
                    "global step %d, epoch: %d, batch: %d, loss: %.5f, accu: %.5f, speed: %.2f step/s"
                    % (global_step, epoch, step, loss, acc,
                       10 / (time.time() - tic_train)))
                tic_train = time.time()
            if args.use_amp:
                scaler.scale(loss).backward()
                scaler.minimize(optimizer, loss)
            else:
                loss.backward()
                optimizer.step()
            lr_scheduler.step()
            optimizer.clear_grad()

            if global_step % args.valid_steps == 0 and rank == 0:
                evaluate(model, criterion, metric, dev_data_loader)

            if global_step % args.save_steps == 0 and rank == 0:
                save_dir = os.path.join(args.save_dir, "model_%d" % global_step)
                if not os.path.exists(save_dir):
                    os.makedirs(save_dir)
                model._layers.save_pretrained(save_dir)
                tokenizer.save_pretrained(save_dir)


In [13]:
do_train()

INFO:paddle.utils.download:unique_endpoints {''}
[2021-11-28 00:38:55,918] [    INFO] - Already cached /root/.paddlenlp/models/fnet-large/spiece.model


global step 10, epoch: 1, batch: 10, loss: 1.14025, accu: 0.50000, speed: 2.40 step/s
global step 20, epoch: 1, batch: 20, loss: 0.63458, accu: 0.59297, speed: 2.15 step/s
global step 30, epoch: 1, batch: 30, loss: 0.70803, accu: 0.62448, speed: 2.42 step/s
global step 40, epoch: 1, batch: 40, loss: 0.79939, accu: 0.61680, speed: 2.27 step/s
global step 50, epoch: 1, batch: 50, loss: 0.61969, accu: 0.63094, speed: 2.52 step/s
global step 60, epoch: 1, batch: 60, loss: 0.83507, accu: 0.63177, speed: 2.14 step/s
global step 70, epoch: 1, batch: 70, loss: 0.52808, accu: 0.63862, speed: 2.27 step/s
global step 80, epoch: 1, batch: 80, loss: 0.62606, accu: 0.64668, speed: 2.33 step/s
global step 90, epoch: 1, batch: 90, loss: 0.67903, accu: 0.65469, speed: 2.43 step/s
global step 100, epoch: 1, batch: 100, loss: 0.48637, accu: 0.65969, speed: 2.34 step/s
eval loss: 0.61626, accu: 0.69128
global step 110, epoch: 1, batch: 110, loss: 0.65411, accu: 0.70625, speed: 1.02 step/s
global step 120,